In [ ]:
from pasteur.kedro.ipython import *
register_kedro()

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from pasteur.transform.attribute import *

In [ ]:
a = NodeLevel.from_str("[1,2,[asbdasf,4],{6,7}]", nullable=True, ukn_val=1)
a

{'None', '1', ['1', '2', ['asbdasf', '4'], {'6', '7'}]}


In [ ]:
a.height

2


In [ ]:
a.get_groups(1)

[0, 1, 2, 3, [4, 5], [6, 7]]


In [ ]:
a.size

8


In [ ]:
a.get_mapping(a.height)

array([0, 1, 2, 2, 2, 2, 2, 2], dtype=uint8)


In [ ]:
Attribute("tst", {"a": IdxColumn(NodeLevel("cat", [1,3,2,32]))})

Attr[na=0,ukn=0]{'a': Idx{'1', '3', '2', '32'}}


In [ ]:
CatAttribute("tst", [1,2,3,4,5,6], True, "abc")

Attr[na=1,ukn=1]{'tst': Idx{'None', 'abc', '1', '2', '3', '4', '5', '6'}}


In [ ]:
OrdAttribute("tst", [1,2,3,4,5,6], True, "abc")

Attr[na=1,ukn=1]{'tst': Idx{'None', 'abc', ['1', '2', '3', '4', '5', '6']}}


In [ ]:
NumAttribute("tst", 50, True)

Attr[na=1,ukn=0]{'tst': Num[50]}


In [ ]:
from pasteur.transform.base import TimeTransformer
import pandas as pd

In [ ]:
tst_dates = pd.to_datetime(
    {
        "year": 2000,
        "month": 1,
        "day": 1,
        "hour": range(6),
        "minute": [5, 6, 7, 30, 31, pd.NA],
        "second": [1, 2, 3, 30, 40, pd.NA],
    }
)
tst_dates.name = "tst"
tst_dates.index = range(2,8)
tst_dates



2   2000-01-01 00:05:01
3   2000-01-01 01:06:02
4   2000-01-01 02:07:03
5   2000-01-01 03:30:30
6   2000-01-01 04:31:40
7                   NaT
Name: tst, dtype: datetime64[ns]


In [ ]:
trn = TimeTransformer("halfminute", nullable=True)
trn.fit(tst_dates);

In [ ]:
enc = trn.transform(tst_dates)
enc


   tst_time
2        11
3       133
4       255
5       422
6       544
7         0


,tst_time
2,11
3,133
4,255
5,422
6,544
7,0


In [ ]:
mp = next(iter(trn.attr.cols.values())).get_human_values()

In [ ]:
pd.Series(mp)[enc["tst_time"]]


11     00:05:00
133    01:06:00
255    02:07:00
422    03:30:30
544    04:31:30
0          None
dtype: object


In [ ]:
trn.reverse(enc)


2   2000-01-01 00:05:00
3   2000-01-01 01:06:00
4   2000-01-01 02:07:00
5   2000-01-01 03:30:30
6   2000-01-01 04:31:30
7                   NaT
Name: tst, dtype: datetime64[ns]


In [ ]:
pd.Series(pd.NaT, index=enc.index)


2   NaT
3   NaT
4   NaT
5   NaT
6   NaT
7   NaT
dtype: datetime64[ns]


In [ ]:
from pasteur.transform.base import DatetimeTransformer, DateTransformer

In [ ]:
trn = DatetimeTransformer(nullable=True)
trn.fit(tst_dates);

In [ ]:
enc = trn.transform(tst_dates)
enc


   tst_year  tst_week  tst_day  tst_time
2       0.0        52        6         1
3       0.0        52        6         3
4       0.0        52        6         5
5       0.0        52        6         8
6       0.0        52        6        10
7       NaN         0        0         0


,tst_year,tst_week,tst_day,tst_time
2,0.0,52,6,1
3,0.0,52,6,3
4,0.0,52,6,5
5,0.0,52,6,8
6,0.0,52,6,10
7,NaN,0,0,0


In [ ]:
trn.reverse(enc)


2   2000-01-01 00:00:00
3   2000-01-01 01:00:00
4   2000-01-01 02:00:00
5   2000-01-01 03:30:00
6   2000-01-01 04:30:00
7                   NaT
Name: tst, dtype: datetime64[ns]
